In [ ]:
%matplotlib inline
from utils.runner import AgentRuns, Runner
from utils.plotter import PlotData
from traffic.environment import TrafficEnvironment
from traffic.agent import FixedCycleTrafficAgent, QLTrafficAgent, DQLTrafficAgent

plot_data = PlotData(
  [
    'system_total_stopped',
    'system_total_waiting_time',
    'system_mean_waiting_time',
    'system_mean_speed'
  ],
  1,
  100
)

traffic_env = TrafficEnvironment(
  net = 'nets/2way-single-intersection/single-intersection.net.xml',
  rou = 'nets/2way-single-intersection/single-intersection-vhvh.rou.xml',
  seconds = 500,
  delta_time = 10,
  yellow_time = 4,
  min_green = 10,
  max_green = 50
)

agents: dict[str, AgentRuns] = {
  'fixedcycle': {
    'cls': FixedCycleTrafficAgent,
    'color': '#aa0000',
    'configs': [{}]
  },
  'qlearning': {
    'cls': QLTrafficAgent,
    'color': '#00aa00',
    'configs': [
      {
        'alpha': 0.1,
        'gamma': 0.75,
        'init_eps': 1,
        'min_eps': 0.1,
        'decay': 0.9
      },
      {
        'alpha': 0.2,
        'gamma': 0.6,
        'init_eps': 1,
        'min_eps': 0.05,
        'decay': 0.5
      }
    ]
  },
  'deepqlearning': {
    'cls': DQLTrafficAgent,
    'color': '#0000aa',
    'configs': [
      {
        'alpha': 0.1,
        'gamma': 0.75,
        'init_eps': 1,
        'min_eps': 0.1,
        'decay': 1
      },
      {
        'alpha': 0.2,
        'gamma': 0.6,
        'init_eps': 1,
        'min_eps': 0.05,
        'decay': 0.75
      }
    ]
  }
}

runner = Runner(plot_data, traffic_env, agents)

runner.run(runner.learn(), 500, False)